<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_09_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background Subtraction

In [ ]:
!pip install opencv-python opencv-contrib-python numpy matplotlib ffmpeg-python --quiet
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ffmpeg
import os
from IPython.display import HTML, Video
print("OpenCV version:", cv.__version__)


# MOG(Mixture of Gaussians)

In [ ]:
#!wget -q https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O sample
!wget -q https://raw.githubusercontent.com/opencv/opencv/4.x/samples/data/vtest.avi -O sample_video.avi
import cv2 as cv
import numpy as np
import os
fgbg_mog2 = cv.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
fgbg_knn = cv.createBackgroundSubtractorKNN(history=500, dist2Threshold=400.0, detectShadows=True)
cap = cv.VideoCapture('sample_video.avi')
if not cap.isOpened():
    print("Video open failed!")
else:
    # 동영상을 저장할 VideoWriter
    width  = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv.CAP_PROP_FPS)
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter('bgsub_output.avi', fourcc, fps if fps>0 else 30, (width*3, height))
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        fgmask_mog2_ = fgbg_mog2.apply(frame)
        fgmask_knn_  = fgbg_knn.apply(frame)
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
        fgmask_mog2_ = cv.morphologyEx(fgmask_mog2_, cv.MORPH_OPEN, kernel)
        fgmask_knn_  = cv.morphologyEx(fgmask_knn_,  cv.MORPH_OPEN, kernel)

        # Apply the MOG2 mask to the original frame to show only the foreground
        foreground_mog2 = cv.bitwise_and(frame, frame, mask=fgmask_mog2_)

        # 시각화를 위해 세 프레임을 가로로 붙여서 저장
        # 원본 / MOG2 배경 제거된 영상 / KNN 마스크
        mask_knn_color  = cv.cvtColor(fgmask_knn_,  cv.COLOR_GRAY2BGR)
        concat_frame = cv.hconcat([frame, foreground_mog2, mask_knn_color])
        out.write(concat_frame)
    cap.release()
    out.release()
    print("Done. Saved as bgsub_output.avi")
    # Colab에서 보기 위해 mp4 변환
    !ffmpeg -y -i bgsub_output.avi -vcodec libx264 bgsub_output.mp4
    if os.path.exists('bgsub_output.mp4'):
        from IPython.display import Video
        display(Video('bgsub_output.mp4', embed=True))
    else:
        print("mp4 파일 생성 실패!")

# Moving Object Trackiing (객체 추적)

In [ ]:
!wget -q https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O traffic
import cv2 as cv
import numpy as np
import os
cap = cv.VideoCapture('traffic')
if not cap.isOpened():
    print("Video open failed!")
else:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read the first frame!")
        cap.release()
    else:
        h, w, _ = frame.shape
        x, y, w_box, h_box = int(w/2 - 50), int(h/2 - 30), 100, 60
        track_window = (x, y, w_box, h_box)
        # 초기 ROI 히스토그램 계산
        roi = frame[y:y+h_box, x:x+w_box]
        hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
        mask_roi = cv.inRange(hsv_roi, (0, 60, 32), (180, 255, 255))
        roi_hist = cv.calcHist([hsv_roi], [0,1], mask_roi, [180, 256], [0,180, 0,256])
        cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)
        term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)
        # 결과 저장용
        fourcc = cv.VideoWriter_fourcc(*'XVID')
        out = cv.VideoWriter('mean_cam_output.avi', fourcc, 30, (w, h))
        while True:
            ret, frame2 = cap.read()

            if not ret:
                break
            hsv = cv.cvtColor(frame2, cv.COLOR_BGR2HSV)
            back_proj = cv.calcBackProject([hsv], [0,1], roi_hist, [0,180, 0,256], 1)
            # 1) Meanshift
            ret_ms, ms_box = cv.meanShift(back_proj, track_window, term_crit)
            x_ms, y_ms, w_ms, h_ms = ms_box
            cv.rectangle(frame2, (x_ms, y_ms), (x_ms+w_ms, y_ms+h_ms), (255,0,0), 2)
            # 2) Camshift
            ret_cs, cs_box = cv.CamShift(back_proj, track_window, term_crit)
            pts = cv.boxPoints(ret_cs)
            #pts = np.int0(pts)
            pts = pts.astype(int)
            cv.polylines(frame2, [pts], True, (0,255,0), 2)
            # Camshift가 갱신한 window로 업데이트
            track_window = cs_box
            out.write(frame2)
        cap.release()
        out.release()
        print("Done. Saved as mean_cam_output.avi")
        # Colab에서 보기 위해 mp4 변환
        !ffmpeg -y -i mean_cam_output.avi -vcodec libx264 mean_cam_output.mp4
        if os.path.exists('mean_cam_output.mp4'):
            from IPython.display import Video
            display(Video('mean_cam_output.mp4', embed=True))
        else:
            print("mp4 파일 생성 실패!")

In [ ]:
import cv2 as cv
import numpy as np
import os

# 비디오 다운로드
!wget -q https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O traffic

cap = cv.VideoCapture('traffic')
if not cap.isOpened():
    print("Video open failed!")
    exit()

ret, frame = cap.read()
if not ret:
    print("Failed to read the first frame!")
    cap.release()
    exit()

h, w, _ = frame.shape

# ✅ 초기 ROI 수동 조정: 앞쪽 하얀 자동차에 맞춤
x, y, w_box, h_box = 270, 200, 90, 60
track_window = (x, y, w_box, h_box)

# 초기 ROI 히스토그램 계산
roi = frame[y:y+h_box, x:x+w_box]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask_roi = cv.inRange(hsv_roi, (0, 60, 32), (180, 255, 255))
roi_hist = cv.calcHist([hsv_roi], [0,1], mask_roi, [180, 256], [0,180, 0,256])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

# 결과 저장용
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('mean_cam_output.avi', fourcc, 30, (w, h))

while True:
    ret, frame2 = cap.read()
    if not ret:
        break

    hsv = cv.cvtColor(frame2, cv.COLOR_BGR2HSV)
    back_proj = cv.calcBackProject([hsv], [0,1], roi_hist, [0,180, 0,256], 1)

    # MeanShift (Optional)
    ret_ms, ms_box = cv.meanShift(back_proj, track_window, term_crit)
    x_ms, y_ms, w_ms, h_ms = ms_box
    cv.rectangle(frame2, (x_ms, y_ms), (x_ms + w_ms, y_ms + h_ms), (255, 0, 0), 2)

    # CamShift
    ret_cs, cs_box = cv.CamShift(back_proj, track_window, term_crit)
    pts = cv.boxPoints(ret_cs)
    pts = pts.astype(int)
    cv.polylines(frame2, [pts], True, (0, 255, 0), 2)

    # 추적 영역 업데이트
    track_window = cs_box
    out.write(frame2)

cap.release()
out.release()
print("✅ 추적 완료: mean_cam_output.avi 저장됨")

# mp4 변환 및 출력
!ffmpeg -y -i mean_cam_output.avi -vcodec libx264 mean_cam_output.mp4
if os.path.exists('mean_cam_output.mp4'):
    from IPython.display import Video
    display(Video('mean_cam_output.mp4', embed=True))
else:
    print("mp4 파일 생성 실패!")


# KCF (Kernelized Correlation Filters)


In [ ]:
# MIL Tracker 예시 (MOSSE 대신 사용)
!wget -q https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O traffic
import cv2 as cv
import numpy as np
# Use MIL tracker
tracker = cv.TrackerMIL_create()
cap = cv.VideoCapture('traffic')
if not cap.isOpened():
    print("Video open failed!")
else:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read the first frame!")
        cap.release()
    else:
        h, w, _ = frame.shape
        # 초기 바운딩 박스 (수동 설정 또는 객체 검출 결과 사용)
        init_bbox = (int(w/2 - 50), int(h/2 - 30), 100, 60)
        # 트래커 초기화
        ok = tracker.init(frame, init_bbox)

        if not ok:
             print("Failed to initialize tracker")
             cap.release()
        else:
            fourcc = cv.VideoWriter_fourcc(*'XVID')
            out = cv.VideoWriter('mil_output.avi', fourcc, 30, (w, h))

            while True:
                ret, frame2 = cap.read()

                if not ret:
                    break
                # 트래커 업데이트
                ok, bbox = tracker.update(frame2)

                # 결과 그리기
                if ok:
                    (x, y, w_box, h_box) = [int(v) for v in bbox]
                    cv.rectangle(frame2, (x,y), (x+w_box, y+h_box), (0,255,255), 2)
                else:
                    cv.putText(frame2, "Tracking Failure", (50,80), cv.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
                out.write(frame2)
            cap.release()
            out.release()
            print("Done. Saved as mil_output.avi")
            # Colab에서 보기 위해 mp4 변환
            !ffmpeg -y -i mil_output.avi -vcodec libx264 mil_output.mp4
            import os
            if os.path.exists('mil_output.mp4'):
                from IPython.display import Video
                display(Video('mil_output.mp4', embed=True))
            else:
                print("mp4 파일 생성 실패!")

In [ ]:
import cv2 as cv
print("버전:", cv.__version__)

try:
    tracker = cv.TrackerKCF_create()
    print("✅ TrackerKCF 생성 성공!")
except AttributeError:
    try:
        tracker = cv.legacy.TrackerKCF_create()
        print("✅ legacy.TrackerKCF 생성 성공!")
    except AttributeError:
        print("❌ TrackerKCF는 현재 설치된 OpenCV에 없음. 'opencv-contrib-python' 설치 필요!")


In [ ]:
import cv2 as cv
print(dir(cv.legacy))

In [ ]:
import cv2 as cv
import numpy as np
import os

# 1. 모델 파일 다운로드 (working links from a different source)
# Using links from https://github.com/djmv/mobilenet_ssd_opencv
!wget -O mobilenet.prototxt https://raw.githubusercontent.com/djmv/mobilenet_ssd_opencv/master/MobileNetSSD_deploy.prototxt
!wget -O mobilenet.caffemodel https://raw.githubusercontent.com/djmv/mobilenet_ssd_opencv/master/MobileNetSSD_deploy.caffemodel

# 2. 모델 로드 확인
net = cv.dnn.readNetFromCaffe("mobilenet.prototxt", "mobilenet.caffemodel")
if net.empty():
    raise Exception("❌ 모델 로딩 실패! 파일이 비어 있거나 손상됨.")
else:
    print("✅ 모델 로딩 성공!")


# 3. 클래스 정의
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# 4. 비디오 불러오기
!wget -q https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O traffic

cap = cv.VideoCapture("traffic")
if not cap.isOpened():
    raise Exception("❌ 비디오 열기 실패")
else:
    print("✅ 비디오 열기 성공!")

# 5. 객체 탐지 루프 (for Colab display)
# Use IPython.display for showing frames in Colab instead of cv.imshow
from google.colab.patches import cv2_imshow

fourcc = cv.VideoWriter_fourcc(*'XVID')
width  = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv.CAP_PROP_FPS)
out = cv.VideoWriter('detection_output.avi', fourcc, fps if fps > 0 else 30, (width, height))


print("🚀 객체 탐지 시작...")
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w = frame.shape[:2]
        # MobileNet-SSD expects 300x300 input
        blob = cv.dnn.blobFromImage(cv.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            # Filter detections by confidence and class (only 'car' in this case)
            if confidence > 0.5: # Confidence threshold
                idx = int(detections[0, 0, i, 1])
                # Ensure the index is within the bounds of the CLASSES list
                if idx < len(CLASSES):
                    label = CLASSES[idx]
                    # Only draw bounding boxes for 'car'
                    if label == "car":
                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (x1, y1, x2, y2) = box.astype("int")
                        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green bounding box
                        # Display label and confidence
                        label_text = f"{label}: {confidence:.2f}"
                        cv.putText(frame, label_text, (x1, y1 - 5),
                                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Write the frame with detections to the output video
        out.write(frame)

        # Display the frame in Colab (optional, can be slow)
        # cv2_imshow(frame)
        # if cv.waitKey(1) & 0xFF == 27: # Press 'Esc' to exit
        #     break

finally:
    cap.release()
    out.release()
    # cv.destroyAllWindows() # Not needed for cv2_imshow

    print("✅ 객체 탐지 완료!")
    print("Done. Saved as detection_output.avi")

    # Colab에서 보기 위해 mp4 변환
    !ffmpeg -y -i detection_output.avi -vcodec libx264 detection_output.mp4

    import os
    if os.path.exists('detection_output.mp4'):
        from IPython.display import Video
        display(Video('detection_output.mp4', embed=True))
    else:
        print("mp4 파일 생성 실패!")

In [ ]:
import cv2 as cv
print(dir(cv))

In [ ]:
import cv2 as cv
import numpy as np
import os

# Ensure 'mobilenet.prototxt' and 'mobilenet.caffemodel' are uploaded to your Colab session storage
try:
    net = cv.dnn.readNetFromCaffe("mobilenet.prototxt", "mobilenet.caffemodel")
    if net.empty():
        raise Exception("❌ 모델 로딩 실패! 파일이 비어 있거나 손상됨. 파일이 올바르게 업로드되었는지 확인하세요.")
    else:
        print("✅ 모델 로딩 성공!")
except cv.error as e:
    print(f"❌ OpenCV 에러 발생: {e}")
    print("모델 파일을 찾거나 로드하는 데 문제가 발생했습니다. 파일 이름과 경로를 확인하세요.")
    # Exit or handle the error appropriately if model loading fails
    exit() # Exit the cell execution

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# 비디오 열기
!wget -q https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O traffic

cap = cv.VideoCapture('traffic')
ret, frame = cap.read()

if not ret:
    print("❌ 첫 프레임 로딩 실패")
    cap.release()
    exit()

# 자동차 검출 (using the loaded model)
blob = cv.dnn.blobFromImage(cv.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
net.setInput(blob)
detections = net.forward()

cars = []
(h, w) = frame.shape[:2]
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    # Filter detections by confidence and class (only 'car' in this case)
    if confidence > 0.5: # You can adjust this confidence threshold
        idx = int(detections[0, 0, i, 1])
        # Ensure the index is within the bounds of the CLASSES list
        if idx < len(CLASSES):
            label = CLASSES[idx]
            # Only consider 'car' detections
            if label == "car":
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x2, y2) = box.astype("int")
                cars.append((x, y, x2 - x, y2 - y)) # Store bounding box as (x, y, width, height)

# Check if any cars were detected
if not cars:
    print("🚫 첫 프레임에서 자동차 객체를 검출하지 못했습니다. 추적을 시작할 수 없습니다.")
    cap.release()
    exit()

# Use the first detected car's bounding box to initialize tracking
track_window = cars[0]
x, y, w_box, h_box = track_window

# Calculate the histogram of the ROI for CamShift
roi = frame[y:y+h_box, x:x+w_box]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask_roi = cv.inRange(hsv_roi, (0, 60, 32), (180, 255, 255)) # Adjust mask as needed
roi_hist = cv.calcHist([hsv_roi], [0,1], mask_roi, [180,256], [0,180,0,256])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# Set termination criteria for CamShift
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

# Setup VideoWriter for output
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('detection_tracking_output.avi', fourcc, 30, (w, h))

print("🚀 객체 추적 시작 (검출 결과 기반)...")
# Tracking loop
while True:
    ret, frame2 = cap.read()
    if not ret:
        break

    hsv = cv.cvtColor(frame2, cv.COLOR_BGR2HSV)
    back_proj = cv.calcBackProject([hsv], [0,1], roi_hist, [0,180, 0,256], 1)

    # Apply CamShift
    ret_cs, cs_box = cv.CamShift(back_proj, track_window, term_crit)

    # Draw the tracking box
    pts = cv.boxPoints(ret_cs)
    pts = pts.astype(int)
    cv.polylines(frame2, [pts], True, (0, 255, 0), 2) # Green box for tracking

    # Update the tracking window for the next frame
    track_window = cs_box

    out.write(frame2)

# Release resources
cap.release()
out.release()

print("✅ 객체 추적 완료!")
print("Done. Saved as detection_tracking_output.avi")

# Convert to MP4 for Colab display
!ffmpeg -y -i detection_tracking_output.avi -vcodec libx264 detection_tracking_output.mp4

import os
if os.path.exists('detection_tracking_output.mp4'):
    from IPython.display import Video
    display(Video('detection_tracking_output.mp4', embed=True))
else:
    print("mp4 파일 생성 실패!")

In [ ]:
import cv2 as cv
# List all attributes in cv2 that contain "Tracker"
tracker_attributes = [attr for attr in dir(cv) if "Tracker" in attr]
print(tracker_attributes)

# Hough Line Transform

In [ ]:
# 예제: Sudoku 이미지에 대한 Hough Transform
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
# sudoku.png 다운로드
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/samples/data/sudoku.png -O sudoku.png
if not os.path.exists('sudoku.png'):
    print("sudoku.png not found.")
else:
    # 1) 이미지 읽기
    img = cv.imread('sudoku.png')
    if img is None:
        print("Could not read sudoku image.")
    else:
        # 2) 그레이스케일 & Canny 에지
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        edges = cv.Canny(gray, 50, 150, apertureSize=3)
        # 3) 표준 HoughLines
        lines = cv.HoughLines(edges, 1, np.pi/180, 200)
        img_hough = img.copy()
        if lines is not None:
            for i in range(len(lines)):
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                scale = 1000
                x1 = int(x0 + scale * (-b))
                y1 = int(y0 + scale * (a))
                x2 = int(x0 - scale * (-b))
                y2 = int(y0 - scale * (a))
                cv.line(img_hough, (x1, y1), (x2, y2), (0,0,255), 2)
        # 4) 확률적 HoughLinesP
        linesP = cv.HoughLinesP(edges, 1, np.pi/180, 80, minLineLength=50, maxLineGap=10)
        img_houghP = img.copy()
        if linesP is not None:
            for i in range(len(linesP)):
                x1, y1, x2, y2 = linesP[i][0]
                cv.line(img_houghP, (x1, y1), (x2, y2), (0,255,0), 2)
        # 5) 시각화
        plt.figure(figsize=(15,8))
        plt.subplot(2,2,1)
        plt.title('Original')
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        plt.axis('off')
        plt.subplot(2,2,2)
        plt.title('Canny Edges')
        plt.imshow(edges, cmap='gray')
        plt.axis('off')
        plt.subplot(2,2,3)
        plt.title('HoughLines (Standard)')
        plt.imshow(cv.cvtColor(img_hough, cv.COLOR_BGR2RGB))
        plt.axis('off')
        plt.subplot(2,2,4)
        plt.title('HoughLinesP (Probabilistic)')
        plt.imshow(cv.cvtColor(img_houghP, cv.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        print('Done!')

# 추가 실험 : 모폴로지(Morphology) 등


In [ ]:
# 추가 실험 코드 셀 : Morphology + Hough
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
# 여기서는 sudoku.png 재사용 해봄
img2 = cv.imread('sudoku.png')
if img2 is None:
    print('No image found for morphological test.')
else:
    gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    # 모폴로지 연산 : closing (이건 효과적인 것을 찾아볼 것)
    kernel = np.ones((3,3), np.uint8)
    closed = cv.morphologyEx(gray2, cv.MORPH_CLOSE, kernel)
    # Canny 에지 검출
    edges2 = cv.Canny(closed, 50, 150)
    # HoughLinesP 수행
    lines2P = cv.HoughLinesP(edges2, 1, np.pi/180, 80, minLineLength=50, maxLineGap=10)
    img_result = img2.copy()
    if lines2P is not None:
        for i in range(len(lines2P)):
            x1, y1, x2, y2 = lines2P[i][0]
            cv.line(img_result, (x1,y1), (x2,y2), (0,255,0), 2)
    # 시각화
    fig, axs = plt.subplots(1, 3, figsize=(15,5))
    axs[0].imshow(cv.cvtColor(gray2, cv.COLOR_BGR2RGB))
    axs[0].set_title('Original Gray')
    axs[0].axis('off')
    axs[1].imshow(closed, cmap='gray')
    axs[1].set_title('Morphology Closed')
    axs[1].axis('off')
    axs[2].imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    axs[2].set_title('HoughLinesP after Morphology')
    axs[2].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# prompt: . 다른 이미지 실험
# 도로, 건축물, 철로 등 직선이 많은 이미지를 고르고 cv.HoughLines / cv.HoughLinesP 를 적용.
# Canny 파라미터 및 Hough 파라미터( threshold , minLineLength , maxLineGap ) 등을 변경해 가며 결과를 비교.

import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os

# 사용할 이미지 파일 이름 지정
image_file = 'building.jpg'
# image_file = 'railway.jpg' # 만약 철로 이미지를 사용하려면 이 줄 활성화

if not os.path.exists(image_file):
    print(f"{image_file} not found. Please upload {image_file} to your Colab session storage.")
else:
    # 이미지 읽기
    img_orig = cv.imread(image_file)

    if img_orig is None:
        print(f"Could not read {image_file}.")
    else:
        # 그레이스케일 변환
        gray = cv.cvtColor(img_orig, cv.COLOR_BGR2GRAY)

        # Canny 에지 검출 파라미터
        canny_threshold1 = 50
        canny_threshold2 = 150
        canny_aperture_size = 3

        # Canny 에지 검출
        edges = cv.Canny(gray, canny_threshold1, canny_threshold2, apertureSize=canny_aperture_size)

        # HoughLinesP 파라미터 (확률적 허프 변환)
        hough_rho = 1
        hough_theta = np.pi / 180
        hough_threshold = 100 # 직선으로 간주할 최소 교차점 수
        hough_min_line_length = 50 # 검출될 최소 직선 길이
        hough_max_line_gap = 10   # 직선으로 간주할 최대 직선 간격

        # HoughLinesP 적용
        linesP = cv.HoughLinesP(edges, hough_rho, hough_theta, hough_threshold,
                                minLineLength=hough_min_line_length,
                                maxLineGap=hough_max_line_gap)

        img_houghP = img_orig.copy()

        # 검출된 직선 그리기
        if linesP is not None:
            print(f"Detected {len(linesP)} lines using HoughLinesP.")
            for i in range(len(linesP)):
                x1, y1, x2, y2 = linesP[i][0]
                cv.line(img_houghP, (x1, y1), (x2, y2), (0, 255, 0), 2) # 초록색 선

        # 결과 시각화
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(cv.cvtColor(img_orig, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title('Canny Edges')
        plt.imshow(edges, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('HoughLinesP Result')
        plt.imshow(cv.cvtColor(img_houghP, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        print("\n--- Parameter Tuning Example ---")
        # 파라미터 변경 예시: threshold를 낮춰 더 많은 직선 검출
        hough_threshold_low = 50
        linesP_low_thresh = cv.HoughLinesP(edges, hough_rho, hough_theta, hough_threshold_low,
                                          minLineLength=hough_min_line_length,
                                          maxLineGap=hough_max_line_gap)
        img_houghP_low_thresh = img_orig.copy()
        if linesP_low_thresh is not None:
             print(f"Detected {len(linesP_low_thresh)} lines with lower threshold ({hough_threshold_low}).")
             for i in range(len(linesP_low_thresh)):
                x1, y1, x2, y2 = linesP_low_thresh[i][0]
                cv.line(img_houghP_low_thresh, (x1, y1), (x2, y2), (255, 0, 0), 2) # 파란색 선

        # 파라미터 변경 예시: minLineLength를 늘려 짧은 직선 제외
        hough_min_line_length_long = 100
        linesP_long_length = cv.HoughLinesP(edges, hough_rho, hough_theta, hough_threshold,
                                           minLineLength=hough_min_line_length_long,
                                           maxLineGap=hough_max_line_gap)
        img_houghP_long_length = img_orig.copy()
        if linesP_long_length is not None:
             print(f"Detected {len(linesP_long_length)} lines with longer minLineLength ({hough_min_line_length_long}).")
             for i in range(len(linesP_long_length)):
                x1, y1, x2, y2 = linesP_long_length[i][0]
                cv.line(img_houghP_long_length, (x1, y1), (x2, y2), (0, 0, 255), 2) # 빨간색 선


        plt.figure(figsize=(18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f'HoughLinesP (Threshold={hough_threshold})')
        plt.imshow(cv.cvtColor(img_houghP, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f'HoughLinesP (Threshold={hough_threshold_low})')
        plt.imshow(cv.cvtColor(img_houghP_low_thresh, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title(f'HoughLinesP (minLineLength={hough_min_line_length_long})')
        plt.imshow(cv.cvtColor(img_houghP_long_length, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        # Canny 파라미터 변경 예시: 높은 임계값 사용
        canny_threshold1_high = 100
        canny_threshold2_high = 200
        edges_high_thresh = cv.Canny(gray, canny_threshold1_high, canny_threshold2_high, apertureSize=canny_aperture_size)

        linesP_high_canny = cv.HoughLinesP(edges_high_thresh, hough_rho, hough_theta, hough_threshold,
                                          minLineLength=hough_min_line_length,
                                          maxLineGap=hough_max_line_gap)

        img_houghP_high_canny = img_orig.copy()
        if linesP_high_canny is not None:
             print(f"Detected {len(linesP_high_canny)} lines with higher Canny thresholds ({canny_threshold1_high}, {canny_threshold2_high}).")
             for i in range(len(linesP_high_canny)):
                x1, y1, x2, y2 = linesP_high_canny[i][0]
                cv.line(img_houghP_high_canny, (x1, y1), (x2, y2), (0, 255, 255), 2) # 노란색 선

        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.title(f'Canny Edges (Thresholds={canny_threshold1_high}, {canny_threshold2_high})')
        plt.imshow(edges_high_thresh, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title('HoughLinesP after High Canny')
        plt.imshow(cv.cvtColor(img_houghP_high_canny, cv.COLOR_BGR2RGB))
        plt.axis('off')

        plt.tight_layout()
        plt.show()